In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv

def get_item_url(url, query):
    return url + query + "/"


def get_item_page(url):
    resp = requests.get(url)
    if resp.status_code != 200:
        print("Invalid url", resp.url)
        return None
    else:
        return resp.text
    
def get_items(dom):
    soup = BeautifulSoup(dom, "html5lib")
    next_page_url = soup.find("li", "next").a["href"] if soup.find("li", "next") else None
    items = list()
    for div in soup.find_all("div", "search-rst clearfix"):
        item = list()
        name = div.h2.a.text.strip()
        item.append(name)
        price = div.find("span", "num").text
        # 先取得價格字串,再移除其中的非數字部分(以空白字串代替非 0-9的字元)
        price = re.sub(r"[^0-9]", "", price)
        item.append(price)
        if div.find("span", "platform-name"):
            item.append(div.find("span", "platform-name").text.strip())
        else:
            item.append("無")
        items.append(item)
    return items, next_page_url

query = "iPhone 11"
item_url = get_item_url("https://ezprice.com.tw/s/", query)
item_page = get_item_page(item_url)
if item_page:
    items, next_page_url = get_items(item_page)
    next_page = get_item_page(item_url + next_page_url)
    while next_page:
        next_items, next_page_url = get_items(next_page)
        items += next_items
        if next_page_url:
            next_page = get_item_page(item_url + next_page_url)
        else:
            break

print("共 %d 件商品" % len(items))
for item in items:
    print(item)

共 440 件商品
['APPLE  iPhone 11 64GB-紅(預購)', '23890', 'isunfar愛順發3C購物網']
['iphone手機自拍桿通用迷你自拍神器拍照神奇自牌干竿架棍棒杠【雙11大促下單五折】', '780', 'Yahoo超級商城']
['倍思 baseus 180cm iphone 7 8 X 傳輸線 充電線 lightning iOS 11 一年保固', '341', 'momo摩天商城']
['CITY BOSS 11代BOX 磨砂 5.8吋 iPhone X/iX 蘋果 APPLE 手機套 磁扣皮套 側掀/側翻/保護套/手機殼/保護殼/背蓋/支架/卡片夾/可站立', '314', 'momo摩天商城']
['【Apple蘋果 24期0利率 送11倍點數】iPhone SE 32G A1723/A1662  蘋果A9手機 完整盒裝(送貼/套/保固一年/福利品)', '6400', 'momo摩天商城']
['【Apple蘋果 24期0利率 送11倍點數】iPhone SE 64G A1723/A1662 蘋果A9手機 完整盒裝(送貼/套/保固一年/福利品)', '6450', 'momo摩天商城']
['【Apple蘋果 24期0利率 送11倍點數】Apple iPhone 7 128G 蘋果手機 完整盒裝 保固一年(紅色 亮黑 現貨 極新福利品)', '12600', 'momo摩天商城']
['【Apple蘋果 24期0利率 送11倍點數】Apple iphone 8 64G 蘋果手機/完整盒裝 保固一年(送藍牙/退傭機/福利品已拆封)', '18800', 'momo摩天商城']
['【Apple蘋果 分24期0利率 送11倍點數】IPhone 6s plus 128G 蘋果手機 完整盒裝 保固一年(送保護套+鋼化膜 極新/福利品)', '12700', 'momo摩天商城']
['【Apple蘋果 送11倍點數】現貨 IPhone 6 plus 5.5吋 16G 蘋果手機 保固一年 全網最低價(福利品 送保護套+鋼化膜)', '8800', 'momo摩天商城']
['【禾宜精品】防滑墊 止滑墊 SEIWA Hello Kitty KT272 11.9*14公分 iphone 7可用', '199', 'momo摩

In [2]:
with open("ezprice.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(("品項", "價格", "商家"))
    for item in items:
        writer.writerow((column for column in item))

In [3]:
with open("ezprice.csv", "r", encoding="utf-8", newline="") as f:
    reader = csv.DictReader(f)
    for row in reader:
        print(row["品項"], row["價格"], row["商家"])

APPLE  iPhone 11 64GB-紅(預購) 23890 isunfar愛順發3C購物網
iphone手機自拍桿通用迷你自拍神器拍照神奇自牌干竿架棍棒杠【雙11大促下單五折】 780 Yahoo超級商城
倍思 baseus 180cm iphone 7 8 X 傳輸線 充電線 lightning iOS 11 一年保固 341 momo摩天商城
CITY BOSS 11代BOX 磨砂 5.8吋 iPhone X/iX 蘋果 APPLE 手機套 磁扣皮套 側掀/側翻/保護套/手機殼/保護殼/背蓋/支架/卡片夾/可站立 314 momo摩天商城
【Apple蘋果 24期0利率 送11倍點數】iPhone SE 32G A1723/A1662  蘋果A9手機 完整盒裝(送貼/套/保固一年/福利品) 6400 momo摩天商城
【Apple蘋果 24期0利率 送11倍點數】iPhone SE 64G A1723/A1662 蘋果A9手機 完整盒裝(送貼/套/保固一年/福利品) 6450 momo摩天商城
【Apple蘋果 24期0利率 送11倍點數】Apple iPhone 7 128G 蘋果手機 完整盒裝 保固一年(紅色 亮黑 現貨 極新福利品) 12600 momo摩天商城
【Apple蘋果 24期0利率 送11倍點數】Apple iphone 8 64G 蘋果手機/完整盒裝 保固一年(送藍牙/退傭機/福利品已拆封) 18800 momo摩天商城
【Apple蘋果 分24期0利率 送11倍點數】IPhone 6s plus 128G 蘋果手機 完整盒裝 保固一年(送保護套+鋼化膜 極新/福利品) 12700 momo摩天商城
【Apple蘋果 送11倍點數】現貨 IPhone 6 plus 5.5吋 16G 蘋果手機 保固一年 全網最低價(福利品 送保護套+鋼化膜) 8800 momo摩天商城
【禾宜精品】防滑墊 止滑墊 SEIWA Hello Kitty KT272 11.9*14公分 iphone 7可用 199 momo摩天商城
【現貨】BASEUS重低音 免持耳機 入耳式 線控 有線 音樂 藍芽耳機 免持聽筒 iPhone 6s 三星 sony 112K32 259 momo摩天商城
ROCK 一分二 Lightning iPhone 7 8 X 充電